In [3]:
from gensim.models import Word2Vec, KeyedVectors
import gensim.downloader as api

In [7]:
from gensim.models import KeyedVectors
wv = KeyedVectors.load("word2vec-google-news-300.kv", mmap='r')


In [22]:
# find most similar words
wv.most_similar(positive=["king", "woman"], negative=["man"])

[('queen', 0.7118193507194519),
 ('monarch', 0.6189674139022827),
 ('princess', 0.5902431011199951),
 ('crown_prince', 0.5499460697174072),
 ('prince', 0.5377321839332581),
 ('kings', 0.5236844420433044),
 ('Queen_Consort', 0.5235945582389832),
 ('queens', 0.5181134343147278),
 ('sultan', 0.5098593831062317),
 ('monarchy', 0.5087411999702454)]

In [23]:
wv.most_similar(positive=['apple', 'dog', 'car', 'tree', 'house'], negative=["nature"])

[('pine_tree', 0.5472700595855713),
 ('lilac_bush', 0.5438607931137085),
 ('SUV', 0.5334113240242004),
 ('puppy', 0.5291835069656372),
 ('Roadmaster_bicycle', 0.5287821888923645),
 ('persimmon_tree', 0.5238192677497864),
 ('berry_patch', 0.5224582552909851),
 ('holly_bush', 0.5204421281814575),
 ('vehicle', 0.520249605178833),
 ('truck', 0.5199848413467407)]

In [24]:
# Define the analogies
analogies = [
    ('king', 'man', 'woman', 'queen'),  # king - man + woman ~= queen
    ('father', 'man', 'woman', 'mother'),  # father - man + woman ~= mother
    ('brother', 'man', 'woman', 'sister'),
    ('Modi', 'BJP', 'Amit', 'Rahul')  # brother - man + woman ~= sister
]

# Test the analogies
for analogy in analogies:
    word1, word2, word3, expected_word = analogy
    result = wv.most_similar(positive=[word1, word3], negative=[word2], topn=1) 
    result_word, confidence = result[0]
    print(f"{word1} - {word2} + {word3} ~= {result_word} (confidence: {confidence:.4f}) (expected: {expected_word})")
    print()

king - man + woman ~= queen (confidence: 0.7118) (expected: queen)

father - man + woman ~= mother (confidence: 0.8463) (expected: mother)

brother - man + woman ~= sister (confidence: 0.8103) (expected: sister)

Modi - BJP + Amit ~= Rohit (confidence: 0.6148) (expected: Rahul)



In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec

In [14]:
import pandas as pd
df=pd.read_csv("IMDB Dataset.csv")

In [15]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [16]:
#Create embeddings for each review
import numpy as np
def get_embedding(words):
    embedding = [wv[word] for word in words if word in wv]
    return np.mean(embedding, axis=0)

embedding=df['review'].apply(get_embedding)  


In [17]:
data=pd.DataFrame(embedding.tolist())

In [18]:
data

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.189172,0.122229,-0.009467,0.135017,-0.050774,0.020281,-0.097690,-0.056541,-0.034925,0.017471,...,0.070264,0.001872,-0.092388,0.094649,-0.027496,-0.164606,-0.108254,-0.027444,-0.108066,0.150257
1,-0.185499,0.125903,-0.014269,0.130941,-0.055272,0.031238,-0.097224,-0.049698,-0.048992,0.022470,...,0.074339,-0.005252,-0.099289,0.083594,-0.020868,-0.159190,-0.101237,-0.027175,-0.116372,0.150226
2,-0.183011,0.120996,-0.015328,0.139813,-0.051502,0.028181,-0.097988,-0.050185,-0.039776,0.018083,...,0.066613,-0.003633,-0.090437,0.099312,-0.019174,-0.158032,-0.105904,-0.026719,-0.110835,0.143818
3,-0.182364,0.120498,-0.004149,0.133790,-0.042799,0.024321,-0.103811,-0.055815,-0.038609,0.015764,...,0.066914,0.002750,-0.098356,0.090438,-0.026194,-0.160535,-0.104623,-0.016506,-0.110176,0.142951
4,-0.184359,0.118263,-0.018029,0.138713,-0.049747,0.024617,-0.096116,-0.046773,-0.036096,0.016781,...,0.072444,-0.006310,-0.093784,0.084664,-0.016571,-0.158703,-0.100584,-0.018903,-0.111406,0.148305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,-0.184346,0.110517,-0.009493,0.137825,-0.049301,0.025772,-0.085716,-0.048533,-0.034881,0.023293,...,0.071611,0.007729,-0.091943,0.091858,-0.023264,-0.159494,-0.106873,-0.028273,-0.104724,0.141879
49996,-0.186760,0.115703,-0.009027,0.136639,-0.065845,0.015393,-0.095995,-0.063566,-0.043412,0.017125,...,0.069022,0.000746,-0.090551,0.097654,-0.026578,-0.162604,-0.112581,-0.031380,-0.103823,0.149314
49997,-0.190601,0.124897,-0.009440,0.134772,-0.058209,0.019148,-0.098645,-0.054445,-0.035229,0.017722,...,0.069275,-0.000127,-0.091185,0.092066,-0.022009,-0.156251,-0.105004,-0.029470,-0.107264,0.151270
49998,-0.178961,0.118619,-0.009768,0.139581,-0.054063,0.021610,-0.095472,-0.049576,-0.039919,0.016822,...,0.069884,-0.006641,-0.093637,0.094791,-0.023632,-0.158687,-0.104327,-0.021977,-0.111325,0.152515


In [19]:
final_df=pd.concat([df,data],axis=1)
final_df

,review,sentiment,0,1,2,3,4,5,6,7,...,290,291,292,293,294,295,296,297,298,299
0,One of the other reviewers has mentioned that ...,positive,-0.189172,0.122229,-0.009467,0.135017,-0.050774,0.020281,-0.097690,-0.056541,...,0.070264,0.001872,-0.092388,0.094649,-0.027496,-0.164606,-0.108254,-0.027444,-0.108066,0.150257
1,A wonderful little production. <br /><br />The...,positive,-0.185499,0.125903,-0.014269,0.130941,-0.055272,0.031238,-0.097224,-0.049698,...,0.074339,-0.005252,-0.099289,0.083594,-0.020868,-0.159190,-0.101237,-0.027175,-0.116372,0.150226
2,I thought this was a wonderful way to spend ti...,positive,-0.183011,0.120996,-0.015328,0.139813,-0.051502,0.028181,-0.097988,-0.050185,...,0.066613,-0.003633,-0.090437,0.099312,-0.019174,-0.158032,-0.105904,-0.026719,-0.110835,0.143818
3,Basically there's a family where a little boy ...,negative,-0.182364,0.120498,-0.004149,0.133790,-0.042799,0.024321,-0.103811,-0.055815,...,0.066914,0.002750,-0.098356,0.090438,-0.026194,-0.160535,-0.104623,-0.016506,-0.110176,0.142951
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,-0.184359,0.118263,-0.018029,0.138713,-0.049747,0.024617,-0.096116,-0.046773,...,0.072444,-0.006310,-0.093784,0.084664,-0.016571,-0.158703,-0.100584,-0.018903,-0.111406,0.148305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,I thought this movie did a down right good job...,positive,-0.184346,0.110517,-0.009493,0.137825,-0.049301,0.025772,-0.085716,-0.048533,...,0.071611,0.007729,-0.091943,0.091858,-0.023264,-0.159494,-0.106873,-0.028273,-0.104724,0.141879
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,-0.186760,0.115703,-0.009027,0.136639,-0.065845,0.015393,-0.095995,-0.063566,...,0.069022,0.000746,-0.090551,0.097654,-0.026578,-0.162604,-0.112581,-0.031380,-0.103823,0.149314
49997,I am a Catholic taught in parochial elementary...,negative,-0.190601,0.124897,-0.009440,0.134772,-0.058209,0.019148,-0.098645,-0.054445,...,0.069275,-0.000127,-0.091185,0.092066,-0.022009,-0.156251,-0.105004,-0.029470,-0.107264,0.151270
49998,I'm going to have to disagree with the previou...,negative,-0.178961,0.118619,-0.009768,0.139581,-0.054063,0.021610,-0.095472,-0.049576,...,0.069884,-0.006641,-0.093637,0.094791,-0.023632,-0.158687,-0.104327,-0.021977,-0.111325,0.152515


In [20]:
#Skipgram
#List of lists of tokens
sentences = df['review'].tolist()
sentences

skipgram = Word2Vec(
    sentences=sentences, 
    sg=1,
    vector_size=50,
    window=5,
    min_count=1,
)
skipgram.save("skipgram.model")

In [21]:
import numpy as np
model = Word2Vec.load("skipgram.model")

# Access word vectors
word_vectors = model.wv
def get_embedding(words):
    embedding = [word_vectors[word] for word in words if word in word_vectors]
    return np.mean(embedding, axis=0)

embedding=df['review'].apply(get_embedding)

data=pd.DataFrame(embedding.tolist())
final_df_skip=pd.concat([df,data],axis=1)
final_df_skip

,review,sentiment,0,1,2,3,4,5,6,7,...,40,41,42,43,44,45,46,47,48,49
0,One of the other reviewers has mentioned that ...,positive,-0.092220,0.015900,-0.050078,0.018820,-0.020079,-0.003617,0.028680,-0.166966,...,-0.006177,0.018450,-0.092623,-0.204954,0.213636,-0.051807,0.139175,-0.059806,0.033305,0.144710
1,A wonderful little production. <br /><br />The...,positive,-0.085687,0.018322,-0.047852,0.026277,-0.026986,-0.001545,0.024396,-0.170356,...,-0.001188,0.014378,-0.088817,-0.210630,0.222597,-0.049209,0.142627,-0.049305,0.025052,0.142302
2,I thought this was a wonderful way to spend ti...,positive,-0.086901,0.016693,-0.050055,0.021995,-0.018446,-0.004968,0.020036,-0.173527,...,-0.003011,0.022035,-0.092129,-0.201836,0.213886,-0.045971,0.144558,-0.060097,0.029454,0.139508
3,Basically there's a family where a little boy ...,negative,-0.085685,0.013428,-0.048762,0.017083,-0.022621,0.001757,0.025551,-0.173102,...,0.001926,0.011180,-0.085886,-0.200152,0.216495,-0.054639,0.159447,-0.059752,0.024359,0.134192
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,-0.090202,0.016848,-0.048088,0.024389,-0.024383,-0.000545,0.019890,-0.166410,...,0.004396,0.017030,-0.087002,-0.209090,0.214241,-0.053833,0.144830,-0.055752,0.028570,0.146987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,I thought this movie did a down right good job...,positive,-0.093104,0.020594,-0.052446,0.021017,-0.021486,-0.007104,0.023030,-0.166511,...,0.004528,0.015331,-0.098177,-0.203349,0.218435,-0.049460,0.140883,-0.058177,0.039677,0.137279
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,-0.089128,0.012021,-0.056077,0.026524,-0.023857,-0.007365,0.023828,-0.166132,...,0.004936,0.016145,-0.091270,-0.211768,0.219830,-0.053980,0.142658,-0.055469,0.028316,0.134940
49997,I am a Catholic taught in parochial elementary...,negative,-0.094823,0.019063,-0.048359,0.020597,-0.023157,-0.004376,0.027088,-0.170008,...,-0.000954,0.011364,-0.091823,-0.207052,0.217235,-0.053368,0.140315,-0.054440,0.036652,0.139102
49998,I'm going to have to disagree with the previou...,negative,-0.093003,0.018570,-0.046252,0.024926,-0.019623,-0.005335,0.028952,-0.172691,...,-0.011210,0.021233,-0.090791,-0.216693,0.212553,-0.053352,0.133010,-0.054192,0.035512,0.145784


In [25]:
cbow = Word2Vec(
    sentences=sentences, 
    sg=0,
    vector_size=50,
    window=5,
    min_count=1,
)
cbow.save("cbow.model")
model = Word2Vec.load("cbow.model")

# Access word vectors
word_vectors = model.wv
def get_embedding(words):
    embedding = [word_vectors[word] for word in words if word in word_vectors]
    return np.mean(embedding, axis=0)

embedding=df['review'].apply(get_embedding)

data=pd.DataFrame(embedding.tolist())
final_df_cbow=pd.concat([df,data],axis=1)
final_df_cbow

,review,sentiment,0,1,2,3,4,5,6,7,...,40,41,42,43,44,45,46,47,48,49
0,One of the other reviewers has mentioned that ...,positive,-0.590409,-0.594597,-0.141007,0.275565,-0.756840,0.338048,-0.165876,0.485945,...,0.322164,-0.108669,0.095552,-0.223832,-0.216666,-0.627814,0.007143,-0.006578,0.195080,-0.561895
1,A wonderful little production. <br /><br />The...,positive,-0.515022,-0.559283,-0.106370,0.364832,-0.732046,0.349756,-0.134462,0.491151,...,0.308188,-0.079476,0.084506,-0.170630,-0.238791,-0.595809,0.017723,0.017886,0.193008,-0.474342
2,I thought this was a wonderful way to spend ti...,positive,-0.568865,-0.549144,-0.079059,0.283254,-0.766857,0.337171,-0.161236,0.477588,...,0.277315,-0.086805,0.078497,-0.176374,-0.246083,-0.599242,0.012596,0.000359,0.189866,-0.523267
3,Basically there's a family where a little boy ...,negative,-0.502089,-0.475104,-0.093087,0.307293,-0.589539,0.314877,-0.090253,0.384206,...,0.318675,-0.125112,0.066365,-0.182455,-0.182845,-0.587343,0.006566,0.011530,0.151125,-0.413343
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,-0.537816,-0.525254,-0.113565,0.325928,-0.736313,0.333601,-0.114182,0.486050,...,0.308319,-0.112412,0.083672,-0.183579,-0.235030,-0.624078,0.014556,0.015977,0.189565,-0.499714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,I thought this movie did a down right good job...,positive,-0.587091,-0.588379,-0.158205,0.319331,-0.775849,0.338660,-0.184986,0.511367,...,0.297184,-0.095441,0.034022,-0.178518,-0.212747,-0.647575,0.022580,0.000840,0.230993,-0.599572
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,-0.526341,-0.497332,-0.127846,0.336163,-0.741187,0.359312,-0.113018,0.446386,...,0.294446,-0.056761,0.069449,-0.162466,-0.234655,-0.592249,0.020062,0.055586,0.144801,-0.482014
49997,I am a Catholic taught in parochial elementary...,negative,-0.573574,-0.601179,-0.142303,0.316527,-0.779688,0.337561,-0.139144,0.507149,...,0.325870,-0.085827,0.075006,-0.208971,-0.217293,-0.643056,0.031372,0.033111,0.204717,-0.598086
49998,I'm going to have to disagree with the previou...,negative,-0.672837,-0.658110,-0.118394,0.256404,-0.849792,0.370652,-0.161664,0.550668,...,0.288171,-0.114124,0.122001,-0.208696,-0.285581,-0.636688,0.013353,-0.016684,0.225425,-0.599796


In [26]:
#Train test splits for all 3 dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(final_df.iloc[:,2:], final_df['sentiment'], test_size=0.2)
X_train_skip, X_test_skip, y_train_skip, y_test_skip = train_test_split(final_df_skip.iloc[:,2:], final_df_skip['sentiment'], test_size=0.2)
X_train_cbow, X_test_cbow, y_train_cbow, y_test_cbow = train_test_split(final_df_cbow.iloc[:,2:], final_df_cbow['sentiment'], test_size=0.2)

In [27]:
#Train the model
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier()
clf.fit(X_train,y_train)
clf_skip=RandomForestClassifier()
clf_skip.fit(X_train_skip,y_train_skip)
clf_cbow=RandomForestClassifier()
clf_cbow.fit(X_train_cbow,y_train_cbow)

RandomForestClassifier()

In [30]:
from sklearn.metrics import accuracy_score, f1_score

# Model 1
y_pred = clf.predict(X_test)
print("Model 1 Accuracy:", accuracy_score(y_test, y_pred))
print("Model 1 F1 Score:", f1_score(y_test, y_pred, average='weighted'))

# Model 2
y_pred_skip = clf_skip.predict(X_test_skip)
print("Model 2 Accuracy:", accuracy_score(y_test_skip, y_pred_skip))
print("Model 2 F1 Score:", f1_score(y_test_skip, y_pred_skip, average='weighted'))

# Model 3
y_pred_cbow = clf_cbow.predict(X_test_cbow)
print("Model 3 Accuracy:", accuracy_score(y_test_cbow, y_pred_cbow))
print("Model 3 F1 Score:", f1_score(y_test_cbow, y_pred_cbow, average='weighted'))

Model 1 Accuracy: 0.6187
Model 1 F1 Score: 0.6187584498522003
Model 2 Accuracy: 0.6278
Model 2 F1 Score: 0.6277401394914471
Model 3 Accuracy: 0.6283
Model 3 F1 Score: 0.6282426673145677
